In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.6 MB/s eta 0:00:00
   ━━

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [ ]:
# Load the training and test data
with open("train.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
with open("test.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Shushant/questionansweringmodel")
model = AutoModelForQuestionAnswering.from_pretrained("Shushant/questionansweringmodel")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/679k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
# Function to prepare data for training
def prepare_data(data):
    inputs = []
    targets = []
    for item in data:
        context = item['context']
        for qas in item['qas']:
            question = qas['question']
            inputs.append(tokenizer(question, context, truncation=True, padding='max_length', max_length=128, return_tensors="pt"))
            # Handle cases where there might be no answers
            if qas['answers']:  # Check if answers list is not empty
                targets.append(qas['answers'][0])
    return inputs, targets

train_inputs, train_targets = prepare_data(train)
test_inputs, test_targets = prepare_data(test)

In [ ]:
# Training loop
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(5):  # Number of epochs
    for i in range(len(train_inputs)):
        optimizer.zero_grad()
        inputs = train_inputs[i]
        # Skip instances without answers
        if i < len(train_targets):
            targets = train_targets[i]
            start_positions = torch.tensor([targets['answer_start']])
            end_positions = torch.tensor([targets['answer_start'] + len(targets['text'])])

            outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
    print(f"Epoch {epoch + 1} completed")



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 completed
Epoch 2 completed
Epoch 3 completed
Epoch 4 completed
Epoch 5 completed


In [ ]:
# Evaluation
model.eval()
total_loss = 0
with torch.no_grad():
    for i in range(len(test_inputs)):
        inputs = test_inputs[i]
        targets = test_targets[i]
        start_positions = torch.tensor([targets['answer_start']])
        end_positions = torch.tensor([targets['answer_start'] + len(targets['text'])])

        outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
        total_loss += outputs.loss.item()
average_loss = total_loss / len(test_inputs)
print(f"Average evaluation loss: {average_loss}")


IndexError: list index out of range

In [ ]:
# Making predictions
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]

for item in to_predict:
    context = item['context']
    for qas in item['qas']:
        question = qas['question']
        inputs = tokenizer(question, context, truncation=True, padding='max_length', max_length=128, return_tensors="pt")
        with torch.no_grad():
             outputs = model(**inputs)
             answer_start = torch.argmax(outputs.start_logits)
             answer_end = torch.argmax(outputs.end_logits) + 1
             answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))
             print(f"Question: {question}")
             print(f"Answer: {answer}")

Question: What is Vin's speciality?
Answer: speciality? [SEP] vin is a mistborn of great power and skill. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
